In [64]:
import json
from pathlib import Path
import pandas as pd
import altair as alt

BASE_DIR = Path.cwd().parent / "participant_runs"

OUTLIER_PARTICIPANTS = {
    "053332"
}

rows = []

for participant_dir in BASE_DIR.iterdir():
    if not participant_dir.is_dir():
        continue

    participant_id = participant_dir.name

    if participant_id in OUTLIER_PARTICIPANTS:
        continue

    for run_dir in participant_dir.iterdir():
        if not run_dir.is_dir():
            continue

        run_name = run_dir.name.lower()

        if run_name.endswith("_mit"):
            tick_condition = "with_tick"
        elif run_name.endswith("_ohne"):
            tick_condition = "without_tick"
        else:
            continue

        for json_file in run_dir.glob("*.json"):
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)

            rows.append({
                "participant_id": participant_id,
                "condition": tick_condition,
                "scene": data.get("sceneName"),
                "hits": data.get("hits"),
                "damage_taken": data.get("damageTaken"),
                "deaths": data.get("deaths"),
                "play_time": data.get("playTime"),
                "file": json_file.name
            })

df = pd.DataFrame(rows)

print(df.head())
print(df.shape)
print("Excluded participants:", OUTLIER_PARTICIPANTS)

  participant_id  condition scene  hits  damage_taken  deaths   play_time  \
0         071225  with_tick  Boss    17            19      33  482.468140   
1         071225  with_tick  Boss    20            22      34  541.671204   
2         071225  with_tick  Boss    24            26      35  626.853455   
3         071225  with_tick  Boss    27            29      36  639.032349   
4         071225  with_tick  Boss    30            32      37  654.742004   

                file  
0  stats_Boss_1.json  
1  stats_Boss_2.json  
2  stats_Boss_3.json  
3  stats_Boss_4.json  
4  stats_Boss_5.json  
(246, 8)
Excluded participants: {'053332'}


In [65]:
run_level = (
    df.groupby(["participant_id", "condition"])
      .agg(
          total_hits=("hits", "max"),
          total_damage=("damage_taken", "max"),
          total_deaths=("deaths", "max"),
          total_play_time=("play_time", "max"),
          n_snapshots=("scene", "count")
      )
      .reset_index()
)

In [66]:
scene_last = (
    df.sort_values("file")
      .groupby(["participant_id", "condition", "scene"])
      .last()
      .reset_index()
)

scene_level = (
    scene_last
    .groupby(["condition", "scene"])
    .agg(
        mean_hits=("hits", "mean"),
        mean_damage=("damage_taken", "mean"),
        mean_deaths=("deaths", "mean"),
        mean_play_time=("play_time", "mean"),
        n=("scene", "count")
    )
    .reset_index()
)

print(scene_level)

      condition        scene  mean_hits  mean_damage  mean_deaths  \
0     with_tick         Boss  25.888889    27.888889    24.888889   
1     with_tick        Lobby  14.555556    16.555556    20.666667   
2     with_tick  StorageRoom   4.111111     6.111111    11.333333   
3     with_tick  Treppenhaus  14.555556    16.555556    20.666667   
4     with_tick     Tutorial   1.333333     3.333333     1.000000   
5  without_tick         Boss  26.444444    28.444444    28.777778   
6  without_tick        Lobby  12.444444    14.444444    24.111111   
7  without_tick  StorageRoom   4.666667     6.666667    16.888889   
8  without_tick  Treppenhaus  12.444444    14.444444    24.111111   
9  without_tick     Tutorial   1.777778     3.777778     1.111111   

   mean_play_time  n  
0      660.900035  9  
1      352.687748  9  
2      209.117159  9  
3      342.160207  9  
4       44.225725  9  
5      648.401723  9  
6      371.602876  9  
7      246.585981  9  
8      367.458237  9  
9       54

In [67]:
pivot = run_level.pivot(index="participant_id", columns="condition")

pivot["delta_deaths"] = (
    pivot["total_deaths"]["with_tick"]
    - pivot["total_deaths"]["without_tick"]
)

pivot["delta_damage"] = (
    pivot["total_damage"]["with_tick"]
    - pivot["total_damage"]["without_tick"]
)

pivot["delta_time"] = (
    pivot["total_play_time"]["with_tick"]
    - pivot["total_play_time"]["without_tick"]
)

delta_df = pivot[["delta_deaths", "delta_damage", "delta_time"]].reset_index()

delta_df.columns = [
    col[0] if isinstance(col, tuple) else col
    for col in delta_df.columns
]

delta_df

,participant_id,delta_deaths,delta_damage,delta_time
0,071225,25,21,310.890778
1,199195,-34,-27,-982.467834
2,232962,-27,-32,-383.117706
3,297088,21,5,356.460419
4,356499,-3,5,20.495148
5,510049,-40,-38,-480.241089
6,807289,-9,-12,-298.859680
7,870227,8,9,275.326782
8,975229,4,10,95.032715


In [68]:
base_deaths = alt.Chart(run_level).encode(
    x=alt.X("condition:N", title="Condition"),
    y=alt.Y("total_deaths:Q", title="Total Deaths"),
    color=alt.Color("condition:N", legend=None)
)

# Boxplot
box_deaths = base_deaths.mark_boxplot()

# Mean als Punkt
mean_deaths = base_deaths.mark_point(
    size=40,
    shape="diamond",
    filled=True,
    color="black"
).encode(
    y=alt.Y("mean(total_deaths):Q"),
    color=alt.value("black"),
    tooltip=[
        alt.Tooltip("condition:N", title="Condition"),
        alt.Tooltip("mean(total_deaths):Q", title="Mean Deaths", format=".2f")
    ]
)

chart_deaths_with_mean = (box_deaths + mean_deaths).properties(
    title="Total Deaths: With vs. Without Tick Loop",
    width=450,
    height=350
)

chart_deaths_with_mean

alt.LayerChart(...)

In [69]:
base_time = alt.Chart(run_level).encode(
    x=alt.X("condition:N", title="Condition"),
    y=alt.Y("total_play_time:Q", title="Total Play Time (s)"),
    color=alt.Color("condition:N", legend=None)
)

# Boxplot
box_time = base_time.mark_boxplot()

# Mean als Punkt
mean_time = base_time.mark_point(
    size=40,
    shape="diamond",
    filled=True,
    color="black"
).encode(
    y=alt.Y("mean(total_play_time):Q"),
    color=alt.value("black"),
    tooltip=[
        alt.Tooltip("condition:N", title="Condition"),
        alt.Tooltip("mean(total_play_time):Q", title="Mean Play Time (s)", format=".1f")
    ]
)

chart_time_with_mean = (box_time + mean_time).properties(
    title="Total Play Time: With vs. Without Tick Loop",
    width=450,
    height=350
)

chart_time_with_mean

alt.LayerChart(...)

In [70]:
chart_delta = alt.Chart(delta_df).mark_bar().encode(
    x=alt.X("participant_id:N", title="Participant"),
    y=alt.Y("delta_deaths:Q", title="Δ Deaths (with - without)"),
    color=alt.condition(
        alt.datum.delta_deaths < 0,
        alt.value("green"),
        alt.value("red")
    )
).properties(
    title="Within-Subject Difference in Deaths"
)

chart_delta

alt.Chart(...)

In [ ]:
delta_long = delta_df.melt(
    id_vars="participant_id",
    value_vars=["delta_deaths", "delta_damage", "delta_time"],
    var_name="metric",
    value_name="delta"
)

chart_delta_facet = alt.Chart(delta_long).mark_bar().encode(
    x=alt.X("participant_id:N", title="Participant"),
    y=alt.Y("delta:Q", title="Δ (with - without)"),
    color=alt.condition(
        alt.datum.delta < 0,
        alt.value("green"),
        alt.value("red")
    ),
    column=alt.Column("metric:N", title="Metric")
).properties(
    title="Within-Subject Differences Across Metrics",
    width=180,
    height=300
).resolve_scale(
    y="independent"
)

chart_delta_facet

alt.Chart(...)

In [72]:
delta_df["baseline_deaths"] = (
    pivot["total_deaths"]["without_tick"].values
)

chart_corr = alt.Chart(delta_df).mark_circle(size=100).encode(
    x=alt.X("baseline_deaths:Q", title="Deaths without Tick"),
    y=alt.Y("delta_deaths:Q", title="Δ Deaths (with − without)"),
    tooltip=["participant_id:N"]
).properties(
    title="Do Weaker Players Benefit More from the Tick Loop?",
    width=450,
    height=350
)

chart_corr

alt.Chart(...)

## Konstruktion des zusammengesetzten Performance-Index

Zur Messung der Gesamtperformance eines Runs wird ein zusammengesetzter Performance-Index gebildet, der drei Aspekte schlechter Spielleistung berücksichtigt: Anzahl der Tode, erlittener Schaden und Gesamtspielzeit.  
Da diese Variablen auf unterschiedlichen Skalen liegen, werden sie zunächst z-standardisiert und anschließend gleichgewichtet gemittelt.

---

### 1) Rohvariablen

Für jeden Run \( i \) seien gegeben:

$$
\begin{align}
D_i &= \text{total\_deaths}_i \\
H_i &= \text{total\_damage}_i \\
T_i &= \text{total\_play\_time}_i
\end{align}
$$

---

### 2) Z-Standardisierung

Jede Variable wird über alle Runs hinweg standardisiert (Mittelwert \(0\), Standardabweichung \(1\)):

$$
\begin{align}
z_{D_i} &= \frac{D_i - \mu_D}{\sigma_D} \\
z_{H_i} &= \frac{H_i - \mu_H}{\sigma_H} \\
z_{T_i} &= \frac{T_i - \mu_T}{\sigma_T}
\end{align}
$$

Dabei sind \( \mu_D, \mu_H, \mu_T \) die Mittelwerte und \( \sigma_D, \sigma_H, \sigma_T \) die Standardabweichungen der jeweiligen Variablen über alle Runs.

---

### 3) Performance-Index als Mittelwert der z-Werte

Der Performance-Index \( P_i \) wird als arithmetisches Mittel der drei z-Werte definiert:

$$
\begin{align}
P_i &= \frac{z_{D_i} + z_{H_i} + z_{T_i}}{3}
\end{align}
$$

---

### 4) Eingesetzte Gesamtformel

Setzt man die z-Transformationen ein, ergibt sich:

$$
\begin{align}
P_i
&= \frac{1}{3}
\left(
\frac{D_i - \mu_D}{\sigma_D}
+ \frac{H_i - \mu_H}{\sigma_H}
+ \frac{T_i - \mu_T}{\sigma_T}
\right)
\end{align}
$$

---

### 5) Interpretation

$$
\begin{align}
P_i < 0 &\Rightarrow \text{bessere Performance als der Durchschnitt} \\
P_i = 0 &\Rightarrow \text{durchschnittliche Performance} \\
P_i > 0 &\Rightarrow \text{schlechtere Performance als der Durchschnitt}
\end{align}
$$

In [73]:
from scipy.stats import zscore


perf_df = run_level.copy()


perf_df["z_deaths"] = zscore(perf_df["total_deaths"])
perf_df["z_damage"] = zscore(perf_df["total_damage"])
perf_df["z_time"] = zscore(perf_df["total_play_time"])


perf_df["performance_index"] = (
perf_df["z_deaths"]
+ perf_df["z_damage"]
+ perf_df["z_time"]
) / 3

In [74]:
base = alt.Chart(perf_df).encode(
    x=alt.X("condition:N", title="Condition"),
    y=alt.Y("performance_index:Q", title="Performance Index (z-score)"),
    color=alt.Color("condition:N", legend=None)
)

# Boxplot
box = base.mark_boxplot()

# Mean als Punkt (korrekt aggregiert!)
mean_points = base.mark_point(
    size=40,
    shape="diamond",
    filled=True,
).encode(
    y=alt.Y("mean(performance_index):Q"),
    color=alt.value("black"),
    tooltip=[
        alt.Tooltip("condition:N", title="Condition"),
        alt.Tooltip("mean(performance_index):Q", title="Mean Performance", format=".2f")
    ]
)

chart_perf_box_with_mean = (box + mean_points).properties(
    title="Composite Performance Index: With vs. Without Tick Loop",
    width=450,
    height=350
)

chart_perf_box_with_mean

alt.LayerChart(...)

In [75]:
pivot_perf = perf_df.pivot(
    index="participant_id",
    columns="condition",
    values="performance_index"
)

pivot_perf["delta_performance"] = (
    pivot_perf["with_tick"] - pivot_perf["without_tick"]
)

delta_perf_df = pivot_perf.reset_index()

chart_perf_delta = alt.Chart(delta_perf_df).mark_bar().encode(
    x=alt.X("participant_id:N", title="Participant"),
    y=alt.Y("delta_performance:Q", title="Δ Performance Index (with − without)"),
    color=alt.condition(
        alt.datum.delta_performance < 0,
        alt.value("green"),
        alt.value("red")
    )
).properties(
    title="Within-Subject Difference in Composite Performance",
    width=500,
    height=300
)

chart_perf_delta

alt.Chart(...)

In [76]:
delta_perf_df["baseline_performance"] = pivot_perf["without_tick"].values

chart_perf_moderation = alt.Chart(delta_perf_df).mark_circle(size=120).encode(
    x=alt.X("baseline_performance:Q", title="Baseline Performance (without Tick)"),
    y=alt.Y("delta_performance:Q", title="Δ Performance (with − without)"),
    tooltip=["participant_id:N"]
).properties(
    title="Do Weaker Players Benefit More? (Composite Performance)",
    width=450,
    height=350
)

chart_perf_moderation

alt.Chart(...)